<a href="https://www.kaggle.com/code/lukaszcichowicz/weekend-trip-advisor-agent?scriptVersionId=235047865" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

#  Weekend Trip Advisor Agent – Capstone Project
####  by Łukasz Cichowicz
### Based on Kaggle 5 Day Intensive GenAI Course with Google.

**Weekend Trip Advisor Agent** is an interactive, generative AI-powered assistant designed to help parents find family-friendly events happening near their location.  
It uses natural language understanding and vector-based semantic search to recommend local weekend activities tailored to user preferences such as travel distance, trip date, interests, and their children's age group.

As a father of two sons, I often find myself wondering how to spend our weekends in a way that's not only exciting and educational, but also a refreshing break from screens — something that helps us create lasting memories together.

That’s how the idea for this project was born: an AI-driven tool that helps parents (like myself!) discover engaging weekend activities for their families.  
By combining generative AI with real-world context, it brings people closer — not just online, but in real life.

This project was developed as part of the **[Gen AI Intensive Course Capstone 2025Q1]**, showcasing multiple core GenAI capabilities through a structured and interactive workflow.




## What's in the Notebook?
 Section 1: Collects user preferences (location, travel range, date, age group) using Gemini parsing

 Section 2: Generates realistic local events near the target location

 Section 3: Filters events by distance (+ tolerance)

 Section 4: Displays structured data for inspection

 Section 5: Stores event embeddings in a vector database using ChromaDB

 Section 6: Gemini agent answers questions using RAG pipeline

 Section 7: Filters events by user-selected themes and children’s age



###  GenAI Capabilities Used:
- **Embeddings** and **Vector Search** (ChromaDB + Gemini)
- **RAG** (querying vector store + Gemini reasoning)
- **Controlled Generation** (JSON-format prompts, filtered responses)
- **Agents** (assistant answering contextual questions)
- **Few-shot prompting** (for Gemini prompts to filter themes and age)

## Let's start:

## Configure API with Google Secrets

We load the Google API key from Kaggle secrets and initialize Gemini.
Also, required libraries like `chromadb` and `pandas` are imported.


In [1]:
# --------------------------------------
# 🔐 Configure API with Google Secrets
# --------------------------------------
!pip install -q chromadb
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai
from datetime import datetime
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
import pandas as pd
import json
import re

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel(model_name="gemini-2.0-flash")

# You may see dependency warnings below (protobuf / google-api-core).
# They do not affect this notebook and can be safely ignored.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.6 MB/s eta 0:00:0

In [2]:

# --------------------------------------
# 🔁 Create shared user context
# --------------------------------------
user_context = {
    "location": None,
    "max_distance_km": None,
    "target_date": None,
    "age_group": None,
    "selected_themes": None,
}

## Step 1 – Collect & Parse User Input (Location, Date, Distance, Age)

We ask the user for input one field at a time: starting location, max distance (km), preferred travel date, and age group of children.

Each input is sent to Gemini to be parsed and standardized into structured format (e.g., parse "next weekend" into a date).

### ❗ Note:
### Normally, the assistant would interactively ask the user for input using `input()`.
### However, because Kaggle notebooks do not support interactive inputs,
### we define the input values manually below.
### These values simulate what a user might enter in a real-world scenario.

In [3]:

# --------------------------------------
# 1. Ask user for location, distance, date, age (individually)
# --------------------------------------
def parse_single_input_with_gemini(field_name: str, user_input: str) -> str:
    today = datetime.today().date()
    prompt = f"""
Convert the user input into structured format. Today is {today}.
Field: {field_name}
Input: {user_input}
If it's a date-related field, return date in format YYYY-MM-DD.
If it's a distance, return only the number.
If it's a location, return title-cased place name.
If it's age, convert age or age range to one of: toddlers, school-age, teenagers.
Only return the cleaned value.
"""
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"⚠️ Could not parse {field_name} with Gemini. Error:", e)
        return None

# Manually set user input (works with Kaggle)
location_input = "Szczecin"
distance_input = "100"
date_input = "next weekend"
age_input = "8 and 13"


user_context["location"] = parse_single_input_with_gemini("location", location_input) or "Szczecin"
user_context["max_distance_km"] = int(parse_single_input_with_gemini("max_distance_km", distance_input) or 100)
user_context["target_date"] = parse_single_input_with_gemini("target_date", date_input) or str(datetime.today().date())
user_context["age_group"] = parse_single_input_with_gemini("age_group", age_input) or "school-age"

print("\n📍 Starting point:", user_context["location"])
print("📏 Max distance:", user_context["max_distance_km"], "km")
print("📅 Travel date:", user_context["target_date"])
print("👨‍👩‍👧‍👦 Age group:", user_context["age_group"])



📍 Starting point: Szczecin
📏 Max distance: 100 km
📅 Travel date: 2025-04-26
👨‍👩‍👧‍👦 Age group: school-age


##  Step 2 – Simulated Events near Target Location

In a real-world solution, events would be retrieved using APIs like Google Places, Eventbrite, or Ticketmaster.  
However, due to authentication limits, API quotas, and to make this notebook self-contained and runnable by any Kaggle user, we simulate this part using Gemini itself.

The generative model is asked to create 4 plausible, family-friendly weekend events based on the user's location and planned date.  
If the model fails (e.g., due to quota or connectivity issues), a fallback static list is used.

This also demonstrates Gemini's capability to generate structured JSON from natural prompts, which is crucial for agent workflows.


In [4]:
# --------------------------------------
# 2. Generate fake events near given location
# --------------------------------------

def generate_sample_events_with_gemini(location, date):
    prompt = f"""
Create 4 family-friendly events happening near {location} on or around {date}.
Include a mix of themes like history, music, science, handmade.
Format as JSON list with fields: name, location, distance_km, theme, description, datetime.
Use town names different than {location}, but nearby.
Don't include any explanations or extra text – only return valid JSON.
"""

    try:
        response = model.generate_content(prompt)
        raw = response.text.strip()

        match = re.search(r"\[\s*{.*?}\s*]", raw, re.DOTALL)
        if match:
            return json.loads(match.group(0))
        else:
            raise ValueError(" No valid JSON list found.")

    except Exception as e:
        print("⚠️ Failed to fetch realistic events. Using fallback.")
        print("Reason:", e)
        return [
            {"name": "Handicraft Market", "location": "Elm Grove", "distance_km": 95, "theme": "Handmade / Fair", "description": "A local market with handmade goods, ceramics, food, and live music.", "datetime": f"{date} 10:00"},
            {"name": "Alternative Music Festival", "location": "Pine Valley", "distance_km": 108, "theme": "Music", "description": "A 3-day festival featuring bands from the region and abroad.", "datetime": f"{date} 12:00"},
            {"name": "Guided City Walk", "location": "Stone Ridge", "distance_km": 115, "theme": "History / Culture", "description": "A guided walk through the town center with stories about its past.", "datetime": f"{date} 14:00"},
            {"name": "Family Science Picnic", "location": "Maple Hill", "distance_km": 120, "theme": "Science / Kids", "description": "Experiments, chemical shows, and games for kids and parents.", "datetime": f"{date} 16:00"},
        ]

sample_events = generate_sample_events_with_gemini(user_context["location"], user_context["target_date"])


## Step 3 – Filter Events by Distance

Events are filtered into three categories:
- 🟢 Within user range
- 🚫 Slightly outside (+20 km tolerance)
- 🔴 Too far


In [5]:
# --------------------------------------
# 3. Distance Filtering (with tolerance)
# --------------------------------------
max_with_tolerance = user_context["max_distance_km"] + 20

for event in sample_events:
    if event["distance_km"] <= user_context["max_distance_km"]:
        event["range_flag"] = "🟢 within range"
    elif event["distance_km"] <= max_with_tolerance:
        event["range_flag"] = "🚫 slightly outside"
    else:
        event["range_flag"] = "🔴 too far"


## Step 4 – Preview Events

We convert the event list into a sorted DataFrame for preview.


In [6]:
# --------------------------------------
# 4. Data Preview (for vector store and agent)
# --------------------------------------
df = pd.DataFrame(sample_events)
df = df[["name", "location", "distance_km", "range_flag", "datetime", "theme", "description"]]
df.sort_values("distance_km", inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()


,name,location,distance_km,range_flag,datetime,theme,description
0,Science Fun Day in Police,Police Cultural Center,20,🟢 within range,2025-04-27T11:00:00,Science,Explore the wonders of science at the Police C...
1,Spring Music Festival in Gryfino,Gryfino Amphitheater,25,🟢 within range,2025-04-26T14:00:00,Music,Celebrate the arrival of spring with a vibrant...
2,Medieval Fair in Goleniów,Goleniów Town Square,35,🟢 within range,2025-04-26T10:00:00,History,Step back in time at the Goleniów Medieval Fai...
3,Handmade Market in Stargard,Stargard Old Town Market,40,🟢 within range,2025-04-26T09:00:00,Handmade,Discover unique treasures at the Handmade Mark...


##  Step 5 – Create Embeddings & Store in ChromaDB

Each event is turned into a vector using Gemini’s embedding model. We store them in ChromaDB to later support semantic search.


In [7]:
# --------------------------------------
# 5. Generate Gemini Embeddings and Store in ChromaDB
# --------------------------------------
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        response = genai.embed_content(
            model="models/embedding-001",
            content=input,
            task_type="retrieval_document",
        )
        if isinstance(input, str):
            return [response["embedding"]]
        return response["embedding"]

content_to_embed = [f"{e['name']} - {e['location']} - {e['theme']} - {e['description']}" for e in sample_events]
ids = [str(i) for i in range(len(sample_events))]

embed_fn = GeminiEmbeddingFunction()
chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(
    name="weekend_events",
    embedding_function=embed_fn
)

vector_store.add(documents=content_to_embed, ids=ids)
print("✅ Embeddings generated and saved to ChromaDB.")

/tmp/ipykernel_13/1916781742.py:18: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  embed_fn = GeminiEmbeddingFunction()


✅ Embeddings generated and saved to ChromaDB.


##  Step 6 – Gemini Assistant: Event Recommendation via RAG

We query the vector store for top matching events, and feed them into Gemini along with user context.

Gemini then returns a personalized recommendation.


In [8]:
# --------------------------------------
# 6. Gemini Agent: Answering questions about weekend plans
# --------------------------------------
def query_vector_store_for_agent(question: str, n_results: int = 2) -> str:
    result = vector_store.query(query_texts=[question], n_results=n_results)
    docs = result["documents"][0]
    joined = "\n\n".join(f"- {doc}" for doc in docs)
    return f"Matching events:\n{joined}"

prompt = f"""
You are a helpful assistant who suggests weekend activities for adults looking to entertain their kids.
You receive short descriptions of local events and respond with friendly recommendations.
Focus on relevance, proximity, and suitability for families.

The user is located in: {user_context['location']} and is willing to travel up to {user_context['max_distance_km']} km.
They plan to go on: {user_context['target_date']}.
"""

context_from_chroma = query_vector_store_for_agent("fun weekend activity with kids")
full_input = prompt + "\n\n" + context_from_chroma

response = model.generate_content(full_input)
print("\n🤖 Gemini Agent says:\n")
print(response.text)


🤖 Gemini Agent says:

Okay, based on your location and the events happening on April 26th, 2025, near Szczecin, here are my recommendations for a fun family weekend:

I think both events sound great, but it really depends on what your kids enjoy!

**Option 1: For the Budding Scientists: Science Fun Day in Police**

*   **Why it's great:** The Science Fun Day in Police at the Police Cultural Center sounds fantastic for kids who love to learn and explore. Hands-on experiments are a great way to keep them engaged and make learning fun. Police is very close to Szczecin (under 20km), making it a very easy and quick trip.

**Option 2: For the Creative Shoppers: Handmade Market in Stargard**

*   **Why it's great:** A trip to the Handmade Market in Stargard offers a chance to discover unique items and support local artists. This would be great if your kids enjoy browsing interesting things, and you might even find some special souvenirs. Stargard is also easily within your 100km travel range

##  Step 7 – Optional Filtering by Theme & Age

User is asked to select themes of interest (e.g., music, science). Each event is re-evaluated by Gemini to check if it matches the selected themes and age group.

Only relevant events are returned and printed.

NOTE: User theme selection is hardcoded due to Kaggle's lack of support for interactive input.
      Normally, this would be collected using `input()`.
 


In [9]:
# --------------------------------------
# 7. Filter by Theme and Age (Optional)
# --------------------------------------
def filter_events_by_theme_and_age(events: list[dict], preferred_themes: list[str], age_group: str) -> list[dict]:
    accepted = []
    for event in events:
        prompt = f"""
The user is planning a family weekend trip and is looking for suitable events.

Age group of the children: {age_group}
Preferred themes: {', '.join(preferred_themes)}

Event:
{json.dumps(event)}

Is this event appropriate for this age group and themes?
Answer with a single word: Yes or No.
"""
        try:
            response = model.generate_content(prompt)
            if "yes" in response.text.strip().lower():
                accepted.append(event)
        except Exception:
            continue
    return accepted

all_themes = list(set(e["theme"].lower() for e in sample_events))
print("\n🎨 Available themes:")
for t in all_themes:
    print(f"- {t}")
# This is where normaly user would be asked to give an input about selected themes...
selected_themes_input = "music, science"
selected_themes = [t.strip().lower() for t in selected_themes_input.split(",") if t.strip() in all_themes]
user_context["selected_themes"] = selected_themes

if not user_context["age_group"]:
    user_context["age_group"] = input("\nWhat is the age group of the children? (e.g., toddlers, school-age, teenagers): ").strip().lower()

sample_events = filter_events_by_theme_and_age(sample_events, user_context["selected_themes"], user_context["age_group"])

print("\n🎯 Final filtered events:")
for e in sample_events:
    print(f"- {e['name']} in {e['location']} on {e['datetime']} | {e['theme']}")



🎨 Available themes:
- history
- handmade
- music
- science

🎯 Final filtered events:
- Science Fun Day in Police in Police Cultural Center on 2025-04-27T11:00:00 | Science
- Spring Music Festival in Gryfino in Gryfino Amphitheater on 2025-04-26T14:00:00 | Music


##  Notes

- For demonstration purposes, event data is generated using Gemini based on user preferences (no real API used).
- The assistant can be extended to connect to real-time APIs (e.g., Eventbrite, Ticketmaster) for live data fetching.
- Some fallback logic is included to handle model failures or vague user inputs.


## License

This Notebook has been released under the Apache 2.0 open source license.